<a href="https://colab.research.google.com/github/gingerchien/QuantHub/blob/main/vectorbt_for_beginners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a compilation of **Chad Thackray's Vectorbt for beginners - Full Python Course** located below and a few of my ideas after learning the course:

https://www.youtube.com/watch?v=JOdEZMcvyac&list=PLKCjdQRzJEHzUc09EhgHD_to93yLnjT0P


#What is Vectorbt?

Accodring to their webstite: https://vectorbt.dev/
vectorbt is a Python package for quantitative analysis that takes a novel approach to backtesting: it operates entirely on pandas and NumPy objects, and is accelerated by Numba to analyze any data at speed and scale. This allows for testing of many thousands of strategies in seconds.

In contrast to other backtesters, vectorbt represents complex data as (structured) NumPy arrays. This enables superfast computation using vectorized operations with NumPy and non-vectorized but dynamically compiled operations with Numba. It also integrates Plotly and Jupyter Widgets to display complex charts and dashboards akin to Tableau right in the Jupyter notebook. Due to high performance, vectorbt can process large amounts of data even without GPU and parallelization and enables the user to interact with data-hungry widgets without significant delays.

# 0. Quickstart

* Documentation: https://vectorbt.dev/api/indicators/basic/
* Github: https://github.com/polakowo/vectorbt

In [3]:
#check python version
!python3 --version

Python 3.10.12


## Install Libraries

In [8]:
#install vectorbt
#!pip3 install vectorbt

#install Yahoo Finance
#!pip3 install yfinance

In [74]:
import vectorbt as vbt
import yfinance as yf
import datetime

## Download Data

In [87]:
end_date = datetime.datetime.now()
start_date= end_date - datetime.timedelta(days=7)
print(start_date)

#btc_price = vbt.YFData.download(['BTC-USD', 'ETH-USD'], interval='1m', start=start_date, end=end_date ,missing_index='drop').get('Close')
btc_price = vbt.YFData.download('BTC-USD', missing_index='drop').get('Close')

2023-12-13 23:47:51.017050


In [88]:
print(btc_price.tail())
print(btc_price.head())
print(type(btc_price))

Date
2023-12-16 00:00:00+00:00    42240.117188
2023-12-17 00:00:00+00:00    41364.664062
2023-12-18 00:00:00+00:00    42623.539062
2023-12-19 00:00:00+00:00    42270.527344
2023-12-20 00:00:00+00:00    43633.664062
Freq: D, Name: Close, dtype: float64
Date
2014-09-17 00:00:00+00:00    457.334015
2014-09-18 00:00:00+00:00    424.440002
2014-09-19 00:00:00+00:00    394.795990
2014-09-20 00:00:00+00:00    408.903992
2014-09-21 00:00:00+00:00    398.821014
Freq: D, Name: Close, dtype: float64
<class 'pandas.core.series.Series'>


## Technical Indicators

In [95]:
#rsi = vbt.RSI.run(btc_price, window=[14,21]) different windows
rsi = vbt.RSI.run(btc_price, window=14)
print(rsi.rsi) #to extract the value, call rsi.rsi

Date
2014-09-17 00:00:00+00:00          NaN
2014-09-18 00:00:00+00:00          NaN
2014-09-19 00:00:00+00:00          NaN
2014-09-20 00:00:00+00:00          NaN
2014-09-21 00:00:00+00:00          NaN
                               ...    
2023-12-16 00:00:00+00:00    61.071835
2023-12-17 00:00:00+00:00    55.392152
2023-12-18 00:00:00+00:00    52.656277
2023-12-19 00:00:00+00:00    41.267414
2023-12-20 00:00:00+00:00    49.505046
Freq: D, Name: (14, Close), Length: 3382, dtype: float64


### Turning Indicator into True/Falst Signals for processing with Vectorbt

In [96]:
entries = rsi.rsi_crossed_below(30)
#print(entries.to_string())

In [97]:
exits = rsi.rsi_crossed_above(70)
#print(exits.to_string())

In [98]:
#test with portfolio pf
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

#check what's going on
print(pf.stats())

Start                          2014-09-17 00:00:00+00:00
End                            2023-12-20 00:00:00+00:00
Period                                3382 days 00:00:00
Start Value                                        100.0
End Value                                      79.460413
Total Return [%]                              -20.539587
Benchmark Return [%]                         9440.874425
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                               89.870408
Max Drawdown Duration                 2174 days 00:00:00
Total Trades                                          37
Total Closed Trades                                   37
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   59.459459
Best Trade [%]                                 37.937622
Worst Trade [%]                

In [99]:
print(pf.total_return()) #can extract individual items from stats()

-0.2053958686215688


In [100]:
#plotting results, the most simple plot in vectorbt
pf.plot().show()

# 1. Custom Indicators using the Indicator Factory
* Reference: https://vectorbt.dev/api/indicators/factory/#naive-approach

In [106]:
end_time = datetime.datetime.now()
start_time= end_time - datetime.timedelta(days=3)
btc_price = vbt.YFData.download('BTC-USD', missing_index='drop', start= start_time, end=end_time, interval='1m').get('Close')

In [107]:
print(btc_price)

Datetime
2023-12-18 00:00:00+00:00    41348.203125
2023-12-18 00:01:00+00:00    41377.671875
2023-12-18 00:02:00+00:00    41363.882812
2023-12-18 00:03:00+00:00    41379.984375
2023-12-18 00:04:00+00:00    41379.726562
                                 ...     
2023-12-20 23:51:00+00:00    43631.363281
2023-12-20 23:52:00+00:00    43629.121094
2023-12-20 23:53:00+00:00    43623.109375
2023-12-20 23:55:00+00:00    43637.671875
2023-12-20 23:56:00+00:00    43629.832031
Name: Close, Length: 4202, dtype: float64


## Define Custom Indicator

In [119]:
import numpy as np
def custom_indicator(close, rsi_window=14, ma_window=50):
  rsi = vbt.RSI.run(close, window=rsi_window).rsi.to_numpy() #the .rsi grabs the actual values to turn it into signal later as raw values are not helpful
  ma = vbt.MA.run(close, window=ma_window).ma.to_numpy()#converting to numpy arrays
  #print(rsi)
  #print(ma)
  #create signal
  trend = np.where(rsi>70, -1, 0)
  trend = np.where((rsi<30) & (close<ma) , 1, trend) #only use & version here, AND does not work, for or use |
  #print(trend)
  return trend

#create indicator object so Vectorbt can recognize it for any custom indicators
indicator = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window'],
    output_names=['value']
).from_apply_func(custom_indicator, rsi_window=14, ma_window=50) #provide default values then point it to the custom indictor

#run and get indicator values
res = indicator.run(btc_price, rsi_window=21, ma_window=50)
#print(res.value)
entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

Start                         2023-12-18 00:00:00+00:00
End                           2023-12-20 23:56:00+00:00
Period                                  2 days 22:02:00
Start Value                                       100.0
End Value                                     101.04395
Total Return [%]                                1.04395
Benchmark Return [%]                           5.518085
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               2.841064
Max Drawdown Duration                   1 days 04:48:00
Total Trades                                         29
Total Closed Trades                                  29
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  79.310345
Best Trade [%]                                 0.542534
Worst Trade [%]                               -1

In [120]:
pf.plot().show()

## What if we want a 5 min indicator on the 1 min data?

In [125]:
import numpy as np

end_time = datetime.datetime.now()
start_time= end_time - datetime.timedelta(days=3)
btc_price = vbt.YFData.download(['BTC-USD', 'ETH-USD'], missing_index='drop', start= start_time, end=end_time, interval='1m').get('Close')

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html
# broadcast_axis = 0 , interchangeable w/ 'index', join = 'right' (takes the keys from the right table only (close is the right table, rsi is the left table))
def custom_indicator2(close, rsi_window=14, ma_window=50):
  close_5m = close.resample('5T').last() #converting 1m candles to 5m
  rsi = vbt.RSI.run(close_5m, window=rsi_window).rsi
  rsi, _ = rsi.align(close, broadcast_axis=0, method='ffill', join='right') #this makes the rsi align to the 1 min but at 5 min intervals, where the other 4 min are NAN in between. forward fill helps fill in the gap.

  #convert everything to numpy
  close = close.to_numpy()
  rsi = rsi.to_numpy()

  ma = vbt.MA.run(close, window=ma_window).ma.to_numpy()
  #print(rsi)
  #print(ma)
  #create signal
  trend = np.where(rsi>70, -1, 0)
  trend = np.where((rsi<30) & (close<ma) , 1, trend) #only use & version here, AND does not work, for or use |
  #print(trend)
  return trend

#create indicator object so Vectorbt can recognize it for any custom indicators
indicator2 = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window'],
    output_names=['value']
).from_apply_func(custom_indicator2, rsi_window=14, ma_window=50, keep_pd=True) #keep_pd = True, this ensures everything stays in panda format instead of getting turned into numpy arrays

#run and get indicator values
res = indicator2.run(btc_price, rsi_window=21, ma_window=50)
#print(res.value)
entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

/usr/local/lib/python3.10/dist-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.



Start                         2023-12-18 01:12:00+00:00
End                           2023-12-21 01:08:00+00:00
Period                                  2 days 20:54:00
Start Value                                       100.0
End Value                                     98.042762
Total Return [%]                              -1.957238
Benchmark Return [%]                           2.342389
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               4.230463
Max Drawdown Duration                   2 days 11:46:00
Total Trades                                        4.0
Total Closed Trades                                 3.5
Total Open Trades                                   0.5
Open Trade PnL                                -0.151174
Win Rate [%]                                  29.166667
Best Trade [%]                                 0.779945
Worst Trade [%]                               -1

<ipython-input-125-10d2a17bbfe3>:44: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x7fe9ddd90b80>. Pass column to select a single column/group.



# 2. Hyperparameter Optimization

# 3. Optimization Techniques

# 4. Graphing/Dashboarding

# 5. Order Types

# 6. Avoid Over-fitting